In this kernel, I have implemented the encoder part of the transformer architecture as mentioned in the famous paper: Attention is all you need.(https://arxiv.org/abs/1706.03762).

Many of other codes are adopted from other kernels. For example, loading the embeddings,  load the training and test data and preprocessing, etc. I really appreciate their contributions.

p.s. When I run this locally, I get validation f1-score around 0.688.

Happy transforming!

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# !pip install transformers
# !pip install sentencepiece

## Imports

In [2]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.layers import BatchNormalization, InputSpec, add
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers, activations
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.utils import Sequence

C:\Users\amakr\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\amakr\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\amakr\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


## Some pre-configurations

In [3]:
embed_size = 300 # how big is each word vector
max_features = 45000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 500 # max number of words in a question to use
n_heads = 12 # Number of heads as in Multi-head attention

In [7]:
train_df = pd.read_csv("../Translated/cleaned/train.csv")
test_df = pd.read_csv("../Translated/cleaned/Test.csv")
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (1436, 3)
Test shape :  (620, 2)


In [8]:
model_forward_name = 'Helsinki-NLP/opus-mt-ny-en'
model_backward_name = 'Helsinki-NLP/opus-mt-en-ny'

In [9]:
from transformers import MarianTokenizer, MarianMTModel
import tensorflow as tf

forward_tokenizer = MarianTokenizer.from_pretrained(model_forward_name)
forward_model = MarianMTModel.from_pretrained(model_forward_name)

backward_tokenizer = MarianTokenizer.from_pretrained(model_backward_name)
backward_model = MarianMTModel.from_pretrained(model_backward_name)

In [10]:
# type(MarianTokenizer.from_pretrained(model_forward_name))
type(forward_tokenizer)

transformers.models.marian.tokenization_marian.MarianTokenizer

In [11]:
import re
def for_back(text):
  batch = forward_tokenizer.prepare_seq2seq_batch(text,
                                                max_length=512,
                                                pad_to_max_length=True,
                                                  truncation = True,
                                        return_tensors = 'pt')
  translated = forward_model.generate(**batch)
  output = [forward_tokenizer.decode(t, skip_special_tokens=True) for t in translated]
  batch = backward_tokenizer.prepare_seq2seq_batch(output,
                                                max_length=512,
                                                pad_to_max_length=True,
                                                  truncation = True,
                                        return_tensors = 'pt')
  translated = backward_model.generate(**batch)
  output = [forward_tokenizer.decode(t, skip_special_tokens=True) for t in translated]
  return re.sub(r'▁', ' ',output[0])
tran = []
for text in tqdm(train_df.Text.values.tolist()):
  tran.append(for_back(text))


100%|███████████████████████████████████████████████████████████████████████████| 1436/1436 [19:56:21<00:00, 49.99s/it]


In [16]:
new_train = train_df[['ID','Label']]
new_train['Text'] = tran

In [18]:
new_train.to_csv('back_translated.csv',index = False)

In [29]:
# import re 

# type(output[0])
# len(output)

' Mosiyana ndi ulamuliro wa achichepere ndi atsogoleri a mtsogolo, achichepere ena, onga ngati Lusbabala ndi Maldala, amapeza liwulo osati kokha chifukwa chakuti amayembekezera kukhala, koma kutsogolera ndi nthaŵi yawo. DLILIFA ndimunthu wachichepere, monga ngati Lungde, kansa ya nyama, amene amaimira Jalphafa, mumzinda wa Mzu, kotero kuti ndikhale chitsogozo, ndipo sindikagwidwa ndi Msonkhano wa Pa Yunivesite ya TD.'

In [18]:
type(forward_tokenizer)

NoneType

In [47]:
# def load_and_prec():


# from sklearn.preprocessing import LabelBinarizer
# lb = LabelBinarizer()
# y_train = lb.fit_transform(train_df['Label'])

# y_train = pd.DataFrame(y_train, columns= lb.classes_)
# train_df = pd.concat([train_df, y_train], axis = 1)
# cols_target = train_df.Label.unique().tolist()

## split to train and val
# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=0,shuffle = True) # hahaha
train_X, val_X, train_y , val_y = train_test_split(train_df, train_df[cols_target], test_size=0.1, random_state = 0,stratify = train_df['Label'])

trn_idx = train_y.index.tolist()
val_idx = val_y.index.tolist()


## fill up the missing values
train_X = train_X["Text"].fillna("_##_").values
val_X = val_X["Text"].fillna("_##_").values
test_X = test_df["Text"].fillna("_##_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen,padding = 'post', truncating = 'post')
val_X = pad_sequences(val_X, maxlen=maxlen,padding = 'post', truncating = 'post')
test_X = pad_sequences(test_X, maxlen=maxlen,padding = 'post', truncating = 'post')

## Get the target values
train_y = train_y.values
val_y = val_y.values  

word_index = tokenizer.word_index
#shuffling the data
# np.random.seed(2018)
# trn_idx = np.random.permutation(len(train_X))
# val_idx = np.random.permutation(len(val_X))

# train_X = train_X[trn_idx]
# val_X = val_X[val_idx]
# train_y = train_y[trn_idx]
# val_y = val_y[val_idx]    

#     return train_X, val_X, test_X, train_y, val_y, tokenizer.word_index,val_idx , trn_idx

## Load Embeddings

In [48]:
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in tqdm(word_index.items()):
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in tqdm(word_index.items()):
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.0053247833,0.49346462
    embed_size = all_embs.shape[1]
    print(emb_mean,emb_std,"para")

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in tqdm(word_index.items()):
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

## Scaled Dot-product attention

In [49]:
class DotProdSelfAttention(Layer):
    """The self-attention layer as in 'Attention is all you need'.
    paper reference: https://arxiv.org/abs/1706.03762
    
    """
    def __init__(self, units,
                 activation=None,
                 use_bias=False,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(DotProdSelfAttention, self).__init__(*kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True

    def build(self, input_shape):
        assert len(input_shape) == 3
        input_dim = input_shape[-1]
        # We assume the output-dim of Q, K, V are the same
        self.kernels = dict.fromkeys(['Q', 'K', 'V'])
        for key, _ in self.kernels.items():
            self.kernels[key] = self.add_weight(shape=(input_dim, self.units),
                                                initializer=self.kernel_initializer,
                                                name='kernel_{}'.format(key),
                                                regularizer=self.kernel_regularizer,
                                                constraint=self.kernel_constraint)
        if self.use_bias:
            raise NotImplementedError
        super(DotProdSelfAttention, self).build(input_shape)
        
    def call(self, x):
        Q = K.dot(x, self.kernels['Q'])
        K_mat = K.dot(x, self.kernels['K'])
        V = K.dot(x, self.kernels['V'])
        attention = K.batch_dot(Q, K.permute_dimensions(K_mat, [0, 2, 1]))
        d_k = K.constant(self.units, dtype=K.floatx())
        attention = attention / K.sqrt(d_k)
        attention = K.batch_dot(K.softmax(attention, axis=-1), V)
        return attention
    
    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)
      
        
    

## The Encoder Block

In [50]:
def encoder(input_tensor):
    """One encoder as in Attention Is All You Need
    """
    # Sub-layer 1
    # Multi-Head Attention
    multiheads = []
    d_v = embed_size // n_heads
    for i in range(n_heads):
        multiheads.append(DotProdSelfAttention(d_v)(input_tensor))
    multiheads = concatenate(multiheads, axis=-1)
    multiheads = Dense(embed_size)(multiheads)
    multiheads = Dropout(0.1)(multiheads)
    
    # Residual Connection
    res_con = add([input_tensor, multiheads])
    # Didn't use layer normalization, use Batch Normalization instead here
    # res_con = BatchNormalization(axis=-1)(res_con)
    
    # Sub-layer 2
    # 2 Feed forward layer
    ff1 = Dense(600, activation='relu')(res_con)
    ff2 = Dense(embed_size)(ff1)
    output = add([res_con, ff2])
    # output = BatchNormalization(axis=-1)(output)
    
    return output

## Positional Encoding

In [51]:
# https://github.com/kpot/keras-transformer/blob/master/keras_transformer/position.py
def positional_signal(hidden_size: int, length: int,
                      min_timescale: float = 1.0, max_timescale: float = 1e4):
    """
    Helper function, constructing basic positional encoding.
    The code is partially based on implementation from Tensor2Tensor library
    https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py
    """

    if hidden_size % 2 != 0:
        raise ValueError(
            f"The hidden dimension of the model must be divisible by 2."
            f"Currently it is {hidden_size}")
    position = K.arange(0, length, dtype=K.floatx())
    num_timescales = hidden_size // 2
    log_timescale_increment = K.constant(
        (np.log(float(max_timescale) / float(min_timescale)) /
         (num_timescales - 1)),
        dtype=K.floatx())
    inv_timescales = (
            min_timescale *
            K.exp(K.arange(num_timescales, dtype=K.floatx()) *
                  -log_timescale_increment))
    scaled_time = K.expand_dims(position, 1) * K.expand_dims(inv_timescales, 0)
    signal = K.concatenate([K.sin(scaled_time), K.cos(scaled_time)], axis=1)
    return K.expand_dims(signal, axis=0)

In [52]:
# https://github.com/kpot/keras-transformer/blob/master/keras_transformer/position.py
class AddPositionalEncoding(Layer):
    """
    Injects positional encoding signal described in section 3.5 of the original
    paper "Attention is all you need". Also a base class for more complex
    coordinate encoding described in "Universal Transformers".
    """

    def __init__(self, min_timescale: float = 1.0,
                 max_timescale: float = 1.0e4, **kwargs):
        self.min_timescale = min_timescale
        self.max_timescale = max_timescale
        self.signal = None
        super().__init__(**kwargs)

    def get_config(self):
        config = super().get_config()
        config['min_timescale'] = self.min_timescale
        config['max_timescale'] = self.max_timescale
        return config

    def build(self, input_shape):
        _, length, hidden_size = input_shape
        self.signal = positional_signal(
            hidden_size, length, self.min_timescale, self.max_timescale)
        return super().build(input_shape)

    def call(self, inputs, **kwargs):
        return inputs + self.signal


## Transformer Encoder model

In [95]:
def model_transformer( n_encoder=3):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, trainable=True)(inp)
    # Add positional encoding
    x = AddPositionalEncoding()(x)
#     x = Dropout(0.1)(x)
    for i in range(n_encoder):
        x = encoder(x)
    # These are my own experiments
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
#     conc = Dense(512, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(20, activation="softmax")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])
    return model

In [54]:
LSTM

tensorflow.python.keras.layers.recurrent_v2.LSTM

In [55]:
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class BaseDataGenerator(Sequence):
    """A data generator"""
    def __init__(self, list_IDs, batch_size=64, shuffle=True):
        self.list_IDs = list_IDs
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        """number of steps in one epoch"""
        # Here is the trick
        return len(self.list_IDs) // (self.batch_size * 2**2)

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size: (index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' 
        X = train_X[list_IDs_temp, :]
        y = train_y[list_IDs_temp]
        return X, y

### Train and Predict

Here I used early stopping and model checkpoint to load the best_val model

In [96]:
# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
def train_pred(model, epochs=2):
    # learning schedule callback
#     loss_history = LossHistory()
#     lrate = BatchLRScheduler(step_decay)
#     callbacks_list = [loss_history, lrate]
#     es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
#     model_path = 'keras_models.h5'
#     mc = ModelCheckpoint(filepath=model_path, monitor='val_loss', save_best_only=True)
#     callbacks = [es, mc]
#     train_generator = BaseDataGenerator(list(np.arange(train_X.shape[0])), batch_size=512)
#     model.fit_generator(train_generator,
#                         epochs=epochs,
#                         validation_data=(val_X, val_y),)
#                         callbacks=callbacks)
#     model = load_model(model_path)
    model.fit(train_X, train_y, batch_size=64,
              epochs=epochs,
              validation_split= 0.1)

    pred_val_y = model.predict([val_X], batch_size=64, verbose=0)
    pred_test_y = model.predict([test_X], batch_size=64, verbose=0)
    return pred_val_y, pred_test_y, model

### Main part: load, train, pred and blend 

In [57]:
# train_X, val_X, test_X, train_y, val_y, word_index,val_idx,trn_idx = load_and_prec()
vocab = []
for w,k in word_index.items():
    vocab.append(w)
    if k >= max_features:
        break
# embedding_matrix_1 = load_glove(word_index)
# embedding_matrix_2 = load_fasttext(word_index)
# embedding_matrix_3 = load_para(word_index)

### Create New Embeddings

In [58]:
## Simple average: http://aclweb.org/anthology/N18-2031

# We have presented an argument for averaging as
# a valid meta-embedding technique, and found experimental
# performance to be close to, or in some cases 
# better than that of concatenation, with the
# additional benefit of reduced dimensionality  


## Unweighted DME in https://arxiv.org/pdf/1804.07983.pdf

# “The downside of concatenating embeddings and 
#  giving that as input to an RNN encoder, however,
#  is that the network then quickly becomes inefficient
#  as we combine more and more embeddings.”
  
# embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_2, embedding_matrix_3], axis = 0)
# embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_3], axis = 0)
# np.shape(embedding_matrix)
# model.evaluate(val_X,val_y)

## Train and Predict

Here I am experimenting with 2 encoders, it's not guaranteed to be optimal, you can try out other numbers. Notice that I used epochs = 8

In [59]:
outputs = []
# outputs[0][1]
# type(train_X[0][0])
# val_X
# train_y

In [60]:
# model.evaluate(train_X,train_y)
# train_X.shape
# train_idx

In [99]:
n_encoder = 1
pred_val_y, pred_test_y, model = train_pred(model_transformer(n_encoder=n_encoder), epochs = 42)
# outputs.append([pred_val_y, pred_test_y, 'transformer_enc{}'.format(n_encoder)])

Epoch 1/42
37/37 [==============================] - 177s 5s/step - loss: 3.2516 - accuracy: 0.1389 - val_loss: 2.7759 - val_accuracy: 0.0618
Epoch 2/42
37/37 [==============================] - 173s 5s/step - loss: 2.7282 - accuracy: 0.1553 - val_loss: 2.7273 - val_accuracy: 0.2085
Epoch 3/42
37/37 [==============================] - 172s 5s/step - loss: 2.6584 - accuracy: 0.1574 - val_loss: 2.7784 - val_accuracy: 0.0888
Epoch 4/42
37/37 [==============================] - 174s 5s/step - loss: 2.6327 - accuracy: 0.1746 - val_loss: 2.6486 - val_accuracy: 0.2085
Epoch 5/42
25/37 [===================>..........] - ETA: 53s - loss: 2.5910 - accuracy: 0.1944

KeyboardInterrupt: 

In [62]:
# for thresh in np.arange(0.1, 0.51, 0.01):
#     thresh = np.round(thresh, 2)
#     print("F1 score at threshold {0:.2f} is {1}".format(thresh, metrics.f1_score(val_y, (pred_val_y>thresh).astype(int))))

In [98]:
# pred_test_y = (pred_test_y > 0.42).astype(int)
# test_df = pd.read_csv("../input/test.csv", usecols=["qid"])
# out_df = pd.DataFrame({"qid":test_df["qid"].values})
# out_df['prediction'] = pred_test_y
# out_df.to_csv("submission.csv", index=False)
model.evaluate(val_X,val_y)

9/9 [==============================] - 0s 18ms/step - loss: 1.3256 - accuracy: 0.6354


[1.3255689144134521, 0.6354166865348816]

In [ ]:
# idx = (pred_test_y > 0.42).astype(int)
# test_df = pd.read_csv("../input/test.csv", usecols=["qid"])
# out_df = pd.DataFrame({"qid":test_df["qid"].values})
# out_df['prediction'] = idx

In [23]:
# mylist = out_df[out_df.prediction == 1].index
# for i in mylist:
#     print(i, end=',')

# Stacking
# train_df = pd.concat([train_df,new_train],ignore_index = True)

# Stacking

In [24]:
# dir(model)
# train_df = pd.read_csv("../Translated/cleaned/train.csv")
# test_df = pd.read_csv("../Translated/cleaned/test.csv")
import re
def clean_text(text):
    text = text.lower()
#     text = re.sub(r"what's", "what is ", text)
#     text = re.sub(r"\'s", " ", text)
#     text = re.sub(r"\'ve", " have ", text)
#     text = re.sub(r"can't", "cannot ", text)
#     text = re.sub(r"n't", " not ", text)
#     text = re.sub(r"i'm", "i am ", text)
#     text = re.sub(r"\'re", " are ", text)
#     text = re.sub(r"\'d", " would ", text)
#     text = re.sub(r"\'ll", " will ", text)
#     text = re.sub(r"\'scuse", " excuse ", text)
#     text = re.sub('\W', ' ', text)
    text = re.sub(r",", " ", text) 
    text = re.sub(r"!", " ", text) 
    text = re.sub(r"\(", " ", text) 
    text = re.sub(r"\)", " ", text) 
    text = re.sub(r"\?", " ", text) 
    text = re.sub(r"\s{2,}", " ", text)  
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

# removing stop words
# other_stop_w = pd.read_csv('../Downloaded_notebooks/words_shared_by_all.csv')
# stopw = [item for sublist in other_stop_w.values.tolist() for item in sublist]
# train_df['Text'].apply(lambda x: [item for item in x.split() if item not in stopw])
# test_df['Text'].apply(lambda x: [item for item in x.split() if item not in stopw])

train_df['Text'] = train_df['Text'].map(lambda com : clean_text(com))
test_df['Text'] = test_df['Text'].map(lambda com : clean_text(com))
X_tfidf = train_df.Text
test_X_tfidf = test_df.Text




In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=45000,max_df = 0.5)

X_dtm = vect.fit_transform(X_tfidf).toarray()

test_X_dtm = vect.transform(test_X_tfidf).toarray()

# from sklearn.preprocessing import LabelBinarizer
# lb = LabelBinarizer()
# y_train = lb.fit_transform(train_df['Label'])

# y_train = pd.DataFrame(y_train, columns= lb.classes_)
# # y_train
# cols_target = train_df['Label'].unique().tolist()
# train_df = pd.concat([train_df, y_train], axis = 1)
# # train_df
cols_target = train_df.Label.unique().tolist()

x_train_1, x_val_1, y_train_1, y_val_1 = train_test_split(X_dtm, train_df[cols_target], test_size=0.1, random_state = 5,stratify = train_df['Label'])
train_X_1, val_X_1, train_y_1 , val_y_1 = train_test_split(train_df, train_df[cols_target], test_size=0.1, random_state = 5,stratify = train_df['Label'])

trn_idx = train_y_1.index.tolist()
val_idx = val_y_1.index.tolist()


## fill up the missing values
train_X_1 = train_X_1["Text"].fillna("_##_").values
val_X_1 = val_X_1["Text"].fillna("_##_").values
# test_X_1 = test_df["Text"].fillna("_##_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X_1))
train_X_1 = tokenizer.texts_to_sequences(train_X_1)
val_X_1 = tokenizer.texts_to_sequences(val_X_1)
# test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X_1 = pad_sequences(train_X_1, maxlen=maxlen,padding = 'post', truncating = 'post')
val_X_1 = pad_sequences(val_X_1, maxlen=maxlen,padding = 'post', truncating = 'post')
# test_X = pad_sequences(test_X, maxlen=maxlen,padding = 'post', truncating = 'post')

## Get the target values
train_y_1 = train_y_1.values
val_y_1 = val_y_1.values  


In [42]:
# train_idx = list(set(X_tfidf.index.tolist()) - set(val_idx.tolist()))
# base_model.evaluate(X_dtm[val_idx],train_df.loc[val_idx,cols_target])
# (y_val == val_y).all()

In [43]:
# train_y.shape
# len(train_idx)

In [44]:
from tensorflow import keras
layers = keras.layers
models = keras.models
# Build the model
from keras import backend as K 

# Do some code, e.g. train and save model

K.clear_session()
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

base_model = models.Sequential()
base_model.add(layers.Dense(1000, input_shape=(45000,)))
# model.add(layers.BatchNormalization())
base_model.add(layers.Activation('linear'))
base_model.add(layers.Dropout(0.2))
# model.add(layers.Dense(2048))
# model.add(layers.BatchNormalization())
# model.add(layers.Activation('relu'))
# model.add(layers.Dense(512))
# # model.add(layers.BatchNormalization())
# model.add(layers.Activation('relu'))
# model.add(layers.Dense(128))
# # model.add(layers.BatchNormalization())
# model.add(layers.Activation('relu'))

# model.add(layers.Dropout(drop_ratio))
base_model.add(layers.Dense(20))
base_model.add(layers.Activation('softmax'))

base_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [45]:
# X_dtm[train_idx].shape
history = base_model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    verbose=1,
                   validation_split = 0.1)

Epoch 1/5
37/37 [==============================] - 15s 399ms/step - loss: 2.4361 - accuracy: 0.2727 - val_loss: 2.1090 - val_accuracy: 0.4556
Epoch 2/5
37/37 [==============================] - 14s 369ms/step - loss: 1.1655 - accuracy: 0.7708 - val_loss: 1.5381 - val_accuracy: 0.6139
Epoch 3/5
37/37 [==============================] - 14s 379ms/step - loss: 0.4064 - accuracy: 0.9510 - val_loss: 1.3009 - val_accuracy: 0.6486
Epoch 4/5
37/37 [==============================] - 16s 426ms/step - loss: 0.1271 - accuracy: 0.9905 - val_loss: 1.2279 - val_accuracy: 0.6641
Epoch 5/5
37/37 [==============================] - 14s 381ms/step - loss: 0.0563 - accuracy: 0.9966 - val_loss: 1.1964 - val_accuracy: 0.6680


In [46]:
base_model.evaluate(x_val,y_val)
# train_X.shape
# (y_train == train_y).all()

9/9 [==============================] - 0s 32ms/step - loss: 1.1862 - accuracy: 0.6562


[1.1862280368804932, 0.65625]

In [66]:
base_model.trainable = False
model.trainable = False

In [67]:
x_train.shape

(2584, 45000)

In [91]:
# train_df = pd.read_csv("../Translated/cleaned/train.csv")
# test_df = pd.read_csv("../Translated/cleaned/test.csv")

K.clear_session()
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
x_train, x_val, y_train, y_val = train_test_split(X_dtm, train_df[cols_target], test_size=0.1, random_state = 0,stratify = train_df['Label'])

input_trans = layers.Input(shape=(maxlen,))
input_tf = layers.Input(shape=(45000,))
output_1 = base_model(input_tf,training = False)
output_2 = model(input_trans,training = False)
x = layers.Concatenate()([output_1,output_2])
# x = layers.Dense(1024, activation = 'linear')(x)
x = layers.Dense(512, activation = 'linear')(x)
# x = BatchNormalization()(x)
x = layers.Dense(256, activation = 'sigmoid')(x)
x = layers.Dense(128, activation = 'linear')(x)
# x = BatchNormalization()(x)
outputs = layers.Dense(20, activation="softmax")(x)
super_model = keras.Model(inputs=[input_trans, input_tf], outputs=outputs)
super_model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
history = super_model.fit(
    [train_X_1,x_train_1], y_train_1, batch_size=32, epochs=5, validation_split = 0.1
)

Epoch 1/5
73/73 [==============================] - 10s 133ms/step - loss: 1.5142 - accuracy: 0.6301 - val_loss: 0.6930 - val_accuracy: 0.8919
Epoch 2/5
73/73 [==============================] - 7s 96ms/step - loss: 0.4603 - accuracy: 0.9110 - val_loss: 0.3963 - val_accuracy: 0.9266
Epoch 3/5
73/73 [==============================] - 8s 111ms/step - loss: 0.3219 - accuracy: 0.9295 - val_loss: 0.3561 - val_accuracy: 0.9228
Epoch 4/5
73/73 [==============================] - 8s 109ms/step - loss: 0.2778 - accuracy: 0.9359 - val_loss: 0.3414 - val_accuracy: 0.9266
Epoch 5/5
73/73 [==============================] - 8s 108ms/step - loss: 0.2696 - accuracy: 0.9406 - val_loss: 0.3454 - val_accuracy: 0.9151


In [92]:
super_model.evaluate([val_X_1,x_val_1], y_val_1)

9/9 [==============================] - 0s 45ms/step - loss: 0.3220 - accuracy: 0.9201


[0.32195645570755005, 0.9201388955116272]

#This is how to submit


In [93]:
# (y_val.values == val_y).all()
# y_val.columns == lb.classes_
preds  = super_model.predict([test_X, test_X_dtm])
# lb.inverse_transform(pd.DataFrame(preds,columns = cols_target)[lb.classes_].values)
test_df['Label'] = lb.inverse_transform(pd.DataFrame(preds,columns = cols_target)[lb.classes_].values)

In [94]:
# test_X.shape
sub = test_df[['ID','Label']]
sub.to_csv('submission_keras_stack0079.csv', index = False)

In [85]:
# (sub.Label == pd.read_csv('submission_keras_stack002.csv').Label).sum()
sub

,ID,Label
0,ID_ADHEtjTi,SOCIAL ISSUES
1,ID_AHfJktdQ,RELIGION
2,ID_AUJIHpZr,RELATIONSHIPS
3,ID_AUKYBbIM,SOCIAL ISSUES
4,ID_AZnsVPEi,SOCIAL ISSUES
...,...,...
615,ID_zdpOUWyJ,POLITICS
616,ID_zhnOomuu,RELATIONSHIPS
617,ID_zmWHvBJb,LAW/ORDER
618,ID_zphjdFIb,SOCIAL ISSUES
